In [45]:
import pandas as pd

df = pd.read_csv('caso_full.csv')
df = df[df['state'] == 'SC']
df = df[df['place_type'] == 'city']
df

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1371475,Florianópolis,4205407.0,2020-03-12,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-12,0.0000,0,1,city,SC,2,0
1371477,Florianópolis,4205407.0,2020-03-13,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-13,0.0000,0,2,city,SC,0,0
1371478,Joinville,4209102.0,2020-03-13,202011,597658.0,590466.0,False,False,1,0.16732,2020-03-13,0.0000,0,1,city,SC,1,0
1371480,Florianópolis,4205407.0,2020-03-14,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-14,0.0000,0,3,city,SC,0,0
1371481,Joinville,4209102.0,2020-03-14,202011,597658.0,590466.0,False,False,1,0.16732,2020-03-14,0.0000,0,2,city,SC,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464145,Zortéa,4219853.0,2021-03-15,202111,3398.0,3363.0,False,True,599,17628.01648,2021-03-14,0.0134,8,322,city,SC,0,0
1464146,Água Doce,4200408.0,2021-03-15,202111,7152.0,7145.0,False,True,540,7550.33557,2021-03-14,0.0056,3,317,city,SC,0,0
1464147,Águas Frias,4200556.0,2021-03-15,202111,2354.0,2366.0,False,True,419,17799.49023,2021-03-14,0.0000,0,294,city,SC,0,0
1464148,Águas Mornas,4200606.0,2021-03-15,202111,6559.0,6469.0,False,True,592,9025.76612,2021-03-14,0.0068,4,344,city,SC,0,0


In [46]:
from urllib.request import urlopen
import json
import gzip


with urlopen('https://servicodados.ibge.gov.br/api/v2/malhas/42?resolucao=5&formato=application/vnd.geo+json') as response:
    response = gzip.GzipFile(fileobj=response)
    counties = json.load(response)

for place in counties['features']:
    place['id'] = place['properties']['codarea']

counties['features'][0] 

{'type': 'Feature',
 'properties': {'codarea': '4200101',
  'centroide': [-52.256564545323165, -26.562096247120955]},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-52.1587, -26.4662],
     [-52.0983, -26.4835],
     [-52.0937, -26.5133],
     [-52.0813, -26.5251],
     [-52.0651, -26.523],
     [-52.0291, -26.5468],
     [-52.0283, -26.5933],
     [-52.0417, -26.5988],
     [-52.0672, -26.6781],
     [-52.0861, -26.6867],
     [-52.0886, -26.6687],
     [-52.1334, -26.6851],
     [-52.1476, -26.6717],
     [-52.1449, -26.6928],
     [-52.1245, -26.7249],
     [-52.1716, -26.7242],
     [-52.1847, -26.7132],
     [-52.1801, -26.6803],
     [-52.1987, -26.6631],
     [-52.2337, -26.6803],
     [-52.2549, -26.6758],
     [-52.2808, -26.6437],
     [-52.3399, -26.6678],
     [-52.3565, -26.6598],
     [-52.3681, -26.6714],
     [-52.3932, -26.6838],
     [-52.4129, -26.6581],
     [-52.3927, -26.6375],
     [-52.4148, -26.6143],
     [-52.4183, -26.5863],
     [-52.4594, -26.

In [57]:
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly
import os
import plotly.graph_objects as go
import numpy as np

date_format = "%Y-%m-%d"

cidades = df['city'].unique()

dia_cidade = {}
df_ordered = df.sort_values(by=['date'])
dates = df_ordered['date'].unique()

for d in dates:
    df_day = df_ordered[df_ordered['date'] == d]
    
    for i,r in df_day.iterrows():
        if r['city'] not in dia_cidade:
            dia_cidade[r['city']] = d
            
dia_cidade

values = []

for i,r in df_ordered.iterrows():
    hoje = datetime.strptime(r['date'], date_format)
    dia = datetime.strptime(dates[0], date_format)
    novo = hoje-dia
    values.append({'Dia': novo.days,
                    'Cod. IBGE': r['city_ibge_code'],
                    'Data': hoje,
                   'Confirmados': r['last_available_confirmed'],
                   'Confirmados_rate': r['last_available_confirmed_per_100k_inhabitants'],
                   'Letalidade': r['last_available_death_rate']*100,
                   'Mortes': r['last_available_deaths'],
                   'Município': r['city'],
                   'População': r['estimated_population_2019'],
                   'Mortalidade': (r['last_available_deaths']/r['estimated_population_2019'])*100,
                   'Mortalidade para cada 100k': (r['last_available_deaths']/r['estimated_population_2019'])*100000,
                   'Novos Casos': r['new_confirmed'],
                   'Novos Casos_rate': r['new_confirmed']/r['estimated_population_2019']*100,
                   'Taxa de Novos Casos': r['new_confirmed']/max(r['last_available_confirmed']-r['new_confirmed'],1)*100,
                   'Novos Obitos': r['new_deaths'],
                  })

df_processado = pd.DataFrame(values)
df_processado

,Dia,Cod. IBGE,Data,Confirmados,Confirmados_rate,Letalidade,Mortes,Município,População,Mortalidade,Mortalidade para cada 100k,Novos Casos,Novos Casos_rate,Taxa de Novos Casos,Novos Obitos
0,0,4205407.0,2020-03-12,2,0.39306,0.00,0,Florianópolis,500973.0,0.000000,0.000000,2,0.000399,200.0,0
1,1,4205407.0,2020-03-13,2,0.39306,0.00,0,Florianópolis,500973.0,0.000000,0.000000,0,0.000000,0.0,0
2,1,4209102.0,2020-03-13,1,0.16732,0.00,0,Joinville,590466.0,0.000000,0.000000,1,0.000169,100.0,0
3,2,4205407.0,2020-03-14,2,0.39306,0.00,0,Florianópolis,500973.0,0.000000,0.000000,0,0.000000,0.0,0
4,2,4209102.0,2020-03-14,1,0.16732,0.00,0,Joinville,590466.0,0.000000,0.000000,0,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92302,368,4206207.0,2021-03-15,1638,14148.74320,1.16,19,Gravatal,11501.0,0.165203,165.203026,0,0.000000,0.0,0
92303,368,4206009.0,2021-03-15,1914,13104.20375,0.68,13,Governador Celso Ramos,14471.0,0.089835,89.834842,0,0.000000,0.0,0
92304,368,4205902.0,2021-03-15,5940,8390.66009,1.62,96,Gaspar,69639.0,0.137854,137.853789,0,0.000000,0.0,0
92305,368,4205704.0,2021-03-15,2505,10623.86021,0.60,15,Garopaba,23078.0,0.064997,64.996967,0,0.000000,0.0,0


In [48]:
import cv2
import os

def makeVideo(dir, df, color, color_range, color_scale = "Viridis", code='Cod. IBGE'):
    dir_name = dir
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        os.mkdir(dir_name+'/src')
        os.mkdir(dir_name+'/src/video')
        
    dias = df['Dia'].unique().astype(np.int32)
    for d in dias:
        data = datetime.strptime(dates[0], date_format)
        data = data + timedelta(days=int(d))
        data = str(data.strftime(date_format))
        print(data)
        dft = df[df['Dia'] == d]
        fig = px.choropleth_mapbox(dft, geojson=counties, locations=code, color=dft[color],
                                color_continuous_scale=color_scale,
                                mapbox_style="carto-positron",
                                opacity=0.8,
                                range_color= color_range,
                                hover_name="Município", hover_data=["Confirmados", 'População'],
                                    zoom=6.2, center = {"lat": -27.4, "lon": -51}
                                )

        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title=data)
        fig.update_layout(
            autosize=False,
            width=800,
            height=600,)
        fig.write_image(dir_name+'/src/video/'+data+'.png')
    
    image_folder = dir_name+'/src/video'
    video_name = dir_name+'/'+dir+'.avi'
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 10, (width,height))

    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = .75
    color = (30, 30, 30) 
    thickness = 1

    for image in images:
        img = cv2.imread(os.path.join(image_folder, image))
        img = cv2.putText(img, image[:-4], org, font, fontScale, color, thickness, cv2.LINE_AA) 
        video.write(img)

    cv2.destroyAllWindows()
    video.release()

In [72]:
def makeMapChart(dir, df, color, color_range, color_scale = "Viridis", code='Cod. IBGE'):
    dir_name = dir
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        os.mkdir(dir_name+'/src')

    dias = df['Dia'].unique().astype(np.int32)
    d = dias[-1]
    data = datetime.strptime(dates[0], date_format)
    data = data + timedelta(days=int(d))
    data = str(data.strftime(date_format))
    print(data)
    dft = df[df['Dia'] == d]
    fig = px.choropleth_mapbox(dft, geojson=counties, locations=code, color=dft[color],
                            color_continuous_scale=color_scale,
                            mapbox_style="carto-positron",
                            opacity=0.8,
                            range_color= color_range,
                            hover_name="Município", hover_data=["Novos Casos","Confirmados","Confirmados_rate", "Letalidade", "Novos Obitos", "Mortes", "Mortalidade para cada 100k", 'População'],
                                zoom=6.2, center = {"lat": -27.4, "lon": -51}
                            )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title=data)
    plotly.offline.plot(fig, filename=(dir_name+'/src/'+data+'.html'))

In [73]:
makeMapChart('casos', df_processado, 'Confirmados_rate', [0,17000], color_scale = "Plasma")

2021-03-15


In [51]:
makeVideo('casos', df_processado, 'Confirmados_rate', [0,17000], color_scale = "Plasma")

2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10

In [77]:
makeMapChart('letalidade', df_processado, 'Letalidade', [0,2.42], color_scale = "Plasma")

2021-03-15


In [60]:
makeVideo('letalidade', df_processado, 'Letalidade', [0,2.42], color_scale = "Plasma")

2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10

In [75]:
makeMapChart('mortalidade', df_processado, 'Mortalidade para cada 100k', [0,130], color_scale = "Plasma")

2021-03-15


In [62]:
makeVideo('mortalidade', df_processado, 'Mortalidade para cada 100k', [0,130], color_scale = "Plasma")

2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10

In [67]:
minicipios_interesse = [
    'Florianópolis','Joinville','Balneário Camboriú','São José','Navegantes','Criciúma','Chapecó','Itajaí','Blumenau','Biguaçu'
]
df_interesse = df_processado[df_processado['Município'].isin(minicipios_interesse)]

main_cols = ["Novos Casos","Confirmados","Confirmados_rate", "Letalidade", "Novos Obitos", "Mortes", "Mortalidade para cada 100k", 'População']

In [68]:
def makeLineChart(name, row):
    fig = px.line(df_interesse, x="Data", y=row, color='Município', hover_name="Município", hover_data=main_cols)
    plotly.offline.plot(fig, filename=(name+'.html'))

In [69]:
makeLineChart('avanco_casos', "Confirmados_rate")
makeLineChart('letalidade', "Letalidade")
makeLineChart('mortalidade', "Mortalidade para cada 100k")
